In [4]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install -y fonts-nanum*

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from collections import deque
import os

import requests

In [6]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("headless")
# options.add_argument("window-size=1500,1200")
options.add_argument("no-sandbox")
options.add_argument("disable-dev-shm-usage")
# options.add_argument("disable-gpu")
# options.add_argument("log-level=3")
webdriver = webdriver.Chrome(r'C:\chromedriver',chrome_options=options)


<ipython-input-6-182858b766b7>:9: DeprecationWarning: use options instead of chrome_options
  webdriver = webdriver.Chrome(r'C:\chromedriver',chrome_options=options)


In [ ]:
#스크롤 이동하며 링크 추출하는 함수이다.
#url과, 내보낼 파일 이름이 변수로 들어간다.

def link_extractor(url,file_name):
    webdriver.get(url)
    time.sleep(5)
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body = webdriver.find_element_by_tag_name("body")
    
    #더보기 버튼이 있으면 실행!! 데이터가 너무 적은 카테고리 때문에 예외처리 하였다.
    try:
        btn = webdriver.find_element_by_css_selector('#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div:nth-child(2) > div > button')
        
        #스크롤 높이 차를 이용한 로직
        while True:
            try:
                body.send_keys(Keys.PAGE_UP)
                time.sleep(20)
                webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
                time.sleep(20)

            except:
                print('버튼 안됨/타임오류')
                break
                
            #현재 스크롤의 높이를 기록한다.
            new_height = webdriver.execute_script("return document.body.scrollHeight")
            
            #반복문 탈출 조건! 이전과 현재의 스크롤 높이를 비교하여 같으면 링크 추출로 넘어간다
            if (new_height == last_height):
                # 그런데 데이터가 많은 카테고리인 경우 렉때문에 버튼이 존재하는데도 스크롤이 안내려가는 경우가 있어, 이를 방지한 추가 조건이다.
                if btn:
                    continue
                print("데이터 내보내기 과정으로 이동하겠습니다/")
                break
            last_height = new_height
            

        #스크롤 끝까지 다 내리고 csv로 내보내기
        try:
            atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
            list_href = [a.get_attribute('href') for a in atag]
            df = pd.DataFrame(list_href)
            df.drop_duplicates(ignore_index = True,inplace=True)
            df.to_csv(f'{file_name}.csv')    
        except:
            print('태그 업서요')
            
            
    #더보기 버튼이 없으면! 스크롤 안내리고 바로 데이터 추출       
    except:
        try:
            atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
            list_href = [a.get_attribute('href') for a in atag]
            df = pd.DataFrame(list_href)
            df.drop_duplicates(ignore_index = True,inplace=True)
            df.to_csv(f'{file_name}.csv')    
        except:
            print('태그 업서요')
        


In [21]:
def info_extractor(file_name):

    titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
    n_supporters, likes = [],[]
    durations, target_amounts = [],[]
    descriptions = []
    visited = []
    error = []

    link_list= list(pd.read_csv(file_name+".csv", sep=',').iloc[:,1])
    
    while link_list:
        try:
            #리스트에서 맨 앞에 요소 빼오기
            url = link_list.pop(0)
            #방문한 리스트 저장
            visited.append(url)

            soup = bs(requests.get(url).text, 'html.parser')

            #titles, categories
            category = soup.find('p',class_ = 'title-info').text
            categories.append(category)
            title = soup.find('h2',class_ = 'title').text
            titles.append(title)

            #days_left, percentages, funding_amounts, n_supporters
            info = soup.select_one('#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.wd-ui-campaign-content > div > div.project-state-info > div.state-box').text.split('\n')
            day = info[1]
            days_left.append(day)
            percentage = info[3].split(' ')[0]
            percentages.append(percentage)
            funding = info[4].split(' ')[0]
            funding_amounts.append(funding)
            supporter = info[5].split(' ')[0].split('명')[0]
            n_supporters.append(supporter)

            #company_names 
            company_name = soup.find("div", {"id":"reward-report-content2"})['data-maker-name']
            company_names.append(company_name)

            #likes
            like = soup.select_one('#cntLike').text
            likes.append(like)

            #durations,target_amounts
            info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
            duration = info[2].split(' ')[-1]
            durations.append(duration)
            target_amount = info[1].split(' ')[-1]
            target_amounts.append(target_amount)

            #descriptions
            description = soup.select_one("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")
            descriptions.append(description.text.split('\n')[1].strip())


        except:
            error.append(url)
            continue
    for i in [titles, categories,days_left, percentages, funding_amounts, n_supporters, company_names,likes,durations,target_amounts,description]:
        print(len(i))
        print(i)
        
    data = {
    'categories':categories,
    'title':titles,
    "company_names":company_names,
    'percentages':percentages, 
    'funding_amounts' :funding_amounts, 
    'days_left' :days_left
                } 

    df = pd.DataFrame(data)
    df['n_supporters'] = n_supporters
    df['likes'] = likes
    df['descriptions'] = descriptions
    df['durations'] = durations
    df['target_amounts'] = target_amounts
    
    
    df.to_csv(f'{file_name}에서 추출한 데이터.csv') 

In [20]:
link_extractor(url,"모임"):  # 파일 이름은 자유롭게 설정해주시면 됩니다.

info_extractor("모임")



26
['[영화모임] 한 편의 영화 같은 3개월, 담화관 시즌모임으로 초대합니다!', '[5.0점앵콜] 퇴직/이직 전 진짜 내가 하고 싶은 일을 찾는 곳ㅣ사하라', '퇴직/이직 하기 전, 진짜 내가 하고 싶은 일을 찾는 곳 <사하라>', "이런 소셜살롱 보셨나요?   영화테마살롱 -  컴'온무비", '완소균이부터 코스토리까지 과거의 경험, 미래의 도전을 공유합니다!', '맥주로 만나는 새로운 경험! 비어스픽 프로그램 멤버십', '[온라인앵콜]퇴사/이직/부업/투잡 전 진짜 내가 하고 싶은 일을 찾는 법', '습관의 탄생 by 생각식당 개업 1주년 기념', "라이프쉐어 다이빙 클럽 - ME '나는 나를 얼마나 알고 있을까요?'", '언젠가 홀로 설 당신에게 보내는 "선택" 이야기 [가수 류세라]', '직장인의 딴짓을 돕는 멤버십커뮤니티, 딴짓클럽 ', '[시크릿사파리] 블랙뮤직과 춤을 사랑하는 2030의 신나는 연말파티!', '나는 무엇을 좋아할까요? by 인문,예술 모임공간 CABINET', '"육아는 누구나 처음이라" 양육자를 위한 커뮤니티 <123swing>', "라이프쉐어 다이빙 클럽 - WORK '내게 맞는 일은 무엇일까요?'", "라이프쉐어 다이빙클럽 - LOVE  '내 사랑은 왜 어려울까요?'", '당신의 "퇴근후2시간",인사라운지에서 열리는 특별한 모임에 참여하세요!', "라이프쉐어 다이빙 클럽 - TRAVEL '머무는 삶 vs 떠도는 삶'", '성찰이 필요할 때, 마음을 서예로 만나는 시간 [서예가 인중 이정화]', '두어블, 40,50대를 위한 취미, 배움, 모임 플랫폼 개발 프로젝트', '(12명 정예  여성멤버 모집) 미국 부동산투자 여성사업가  변신프로젝트', "똑똑! 혹시 '귀촌' 생각 여기있나요?", '스트레스 [끊다] 익명 프리미엄 모임 / 3분기 시즌권 / 속시원하게풀자', '떼구르르 보드게임여행', '100여명의 창작자들이 거쳐간 아티스트들을 위한 공간, 아트라우드', '스트라디움 GSML 서포터즈 모집']
26
['모임', '모임

ValueError: arrays must all be same length

NameError: name 'data' is not defined